# Feature selection


In [10]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import os
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
class New_LemmaTokenizer:
     def __init__(self):
       self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
       return [self.wnl.lemmatize(t,pos =get_wordnet_pos(t)) for t in word_tokenize(doc) if t.isalpha()]
class StemTokenizer:
     def __init__(self):
       self.wnl =PorterStemmer()
     def __call__(self, doc):
       return [self.wnl.stem(t) for t in word_tokenize(doc) if t.isalpha()]

class NBN():
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y
    self.topic_portion = []
    self.freq_feature = np.zeros((8,self.X.shape[1]))

  def build_NBN(self):
    topics = np.array([['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']])
    each_topic = []
    topic_portion = []

    for topic in topics:
      each_topic.append((sum([i == topic for i in self.Y]))[0])
      topic_portion.append((sum([i == topic for i in self.Y])/len(self.Y))[0])
    #print(each_topic)
    #print(topic_portion)
    self.topic_portion = topic_portion

    i=0
    j=0
    #maxFeatures = 200
    sum_feature = np.zeros((8,self.X.shape[1]))
    freq_feature = np.zeros((8,self.X.shape[1]))
    for count in each_topic:
      sum_feature[i,:] = np.sum(self.X[j:j+count,:],axis=0)
      freq_feature[i,:] = (np.sum(self.X[j:j+count,:],axis=0)+1)/(count+2)
      #freq_feature[i,:] = (np.sum(trains[j:j+count,:],axis=0))/(count)
      i=i+1
      j=j+count
     
    self.freq_feature = freq_feature
    #print(freq_feature)
    return topic_portion,freq_feature,topics


  def predict(self,X):
    prob = np.ones((1,8))
    for j in np.arange(X.shape[1]):
        prob = np.multiply(np.multiply(np.power(self.freq_feature[:,j],X[0,j]),np.power(1-self.freq_feature[:,j],1-X[0,j])),prob)
    prob = np.log(np.multiply(self.topic_portion,prob))
    category = np.argmax(prob)
    
    return category

def Accu_eval(X,Y,topics):
  n = X.shape[0]
  result = np.zeros((1,n))
  for i in np.arange(n):
    result[0,i] = NBN1.predict(X[i,:])

  #print(result)

  Y_original = []
  correct = 0
  for i in np.arange(n):
    Y_original.append(topics.tolist().index([Y[i]]))
    #Y_original.append(topics.index(Y[i]))
    if topics.tolist().index([Y[i]])-result[0,i] == 0:
      
      correct = correct+1

  #print(Y_original)
  return correct/n, result


def trans_train(X):
  Z = X
  for i in np.arange(X.shape[0]):
    for j in np.arange(X.shape[1]):
      if X[i,j] != 0:
        Z[i,j] = 1
  return Z


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


information_gain

In [16]:

my_stop_words = text.ENGLISH_STOP_WORDS
#import datasets
#train = pd.read_csv('train_no_gamenews_computers.csv')
train = pd.read_csv('train.csv')
n=train.shape[0]
#print(n)
s=10000

skip = sorted(random.sample(range(1,n+1),n-s))
#print(len(skip))
train = pd.read_csv('train.csv',skiprows=skip)
test = pd.read_csv('test.csv')
#print(train.shape[0])

X = pd.DataFrame(train['body'])
Y = pd.DataFrame(train['subreddit'])
X = X['body'].tolist() 
Y = Y['subreddit'].tolist()
Y_ = np.array(Y)
X_ = np.array(X)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
vectorizerfeature = CountVectorizer(min_df =1,binary=True,stop_words = my_stop_words, tokenizer=StemTokenizer())
vectors_train = vectorizerfeature.fit_transform(X_)
label = Y_

from scipy.stats import entropy
import pandas as pd
def information_gain(members, split):

    entropy_before = entropy(members.value_counts(normalize=True))
    split.name = 'split'
    members.name = 'members'
    grouped_distrib = members.groupby(split).value_counts(normalize=True).reset_index(name='count').pivot_table(index='split', columns='members', values='count').fillna(0)
    entropy_after = entropy(grouped_distrib, axis=1)
    entropy_after *= split.value_counts(sort=False, normalize=True)
    return entropy_before - entropy_after.sum()

ig=[]
for i in range(vectors_train.todense().shape[1]):
  x=vectors_train.todense()
  x0 = x[:, i]
  x0=np.array(x0).flatten()
  x00 = pd.Series(x0.tolist())
  label0= pd.Series(label.tolist())
  ig.append(information_gain(label0, x00))
#print(ig)
print(len(ig))
precis = []
f1 = []
reca= []
for feanum in range(5):
  import heapq
  lis= ig.copy()
  n=(feanum+1)*1000
  max_number = heapq.nlargest(n, lis) 
  max_index = []
  for t in max_number:
      index = lis.index(t)
      max_index.append(index)
      lis[index] = 0
      


  voca=[]
  for i in max_index:
    voca.append(vectorizerfeature.get_feature_names()[i])
  voca2 = sorted(set(voca),key=voca.index)

  vectorizer = CountVectorizer(vocabulary=voca2,binary=True,stop_words = my_stop_words, tokenizer=StemTokenizer())
  vectors_train = vectorizer.fit_transform(X_)
  vectors_train.todense()

  kf = KFold(n_splits=6,shuffle=True)
  kf.get_n_splits(X_)
  accuracy = []
  f1_sc = []
  recall_sc= []
  X_train_ = []
  y_train_ = []
  X_test_ = []
  y_test_ = []
  for train_index, test_index in kf.split(X_):

    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = Y_[train_index].tolist(), Y_[test_index].tolist()
    X_train_ = []
    X_test_ = []

    print(type(X_train))
    X_train = X_train.tolist()
    X_test = X_test.tolist()
    vectorizer = CountVectorizer(vocabulary=voca2,binary=True, tokenizer=StemTokenizer())
    vectors_train = vectorizer.fit_transform(X_train)
    vectors_test = vectorizer.transform(X_test)
    print(vectors_train.shape)
    trains = vectors_train.todense()
    tests = vectors_test.todense()
    print(len(tests))
    print(len(y_test))

    NBN1 = NBN(trains, y_train)
    topic_portion,freq_feature,topics = NBN1.build_NBN()
    print(topics.tolist())
    accu, result = Accu_eval(tests,y_test,topics)
    n=len(y_test)
    y_test0=[]
    for i in range(n):
      y_test0.append(topics.tolist().index([y_test[i]]))
    accuracy.append(metrics.precision_score(y_test0, result[0], average='weighted'))
    f1_sc.append(metrics.f1_score(y_test0, result[0], average='weighted'))
    recall_sc.append(metrics.recall_score(y_test0, result[0], average='weighted'))
    print(metrics.classification_report(y_test0, result[0]))
  print(np.mean(accuracy))
  print(np.mean(f1_sc))
  print(np.mean(recall_sc))
  precis.append(np.mean(accuracy))
  f1.append(np.mean(f1_sc))
  reca.append(np.mean(recall_sc))
print('precision_score',precis)
print('f1_score',f1)
print('recall_score',reca)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

1947


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

<class 'numpy.ndarray'>
(83, 10)
17
17
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           2       0.50      1.00      0.67         3
           3       1.00      0.25      0.40         4
           4       0.86      1.00      0.92         6
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         1

    accuracy                           0.59        17
   macro avg       0.39      0.38      0.33        17
weighted avg       0.63      0.59      0.54        17

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarnin

(83, 50)
17
17
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

         0.0       0.50      1.00      0.67         2
         2.0       0.00      0.00      0.00         0
         3.0       1.00      0.67      0.80         3
         4.0       0.80      0.50      0.62         8
         5.0       1.00      1.00      1.00         1
         6.0       1.00      0.33      0.50         3

    accuracy                           0.59        17
   macro avg       0.72      0.58      0.60        17
weighted avg       0.85      0.59      0.66        17

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(84, 50)
16
16
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

           0       1.00      0.86      0.92         7
           1       1.00      0.50      0.67         2
           2       0.00      0.00      0.00         1
           3       0.33      1.00      0.50         1
           4       0.25      1.00      0.40         1
           6       1.00      0.25      0.40         4

    accuracy                           0.62        16
   macro avg       0.60      0.60      0.48        16
weighted avg       0.85      0.62      0.64        16

<class 'numpy.ndarray'>
(84, 50)
16
16
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       1.00      1.00      1.00         2
           2       0.25      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


log likilhood ratio

In [4]:
my_stop_words = text.ENGLISH_STOP_WORDS
#import datasets
#train = pd.read_csv('train_no_gamenews_computers.csv')
train = pd.read_csv('train.csv')
n=train.shape[0]
#print(n)
s=10000

skip = sorted(random.sample(range(1,n+1),n-s))
#print(len(skip))
train = pd.read_csv('train.csv',skiprows=skip)
test = pd.read_csv('test.csv')
#print(train.shape[0])

X = pd.DataFrame(train['body'])
Y = pd.DataFrame(train['subreddit'])
X = X['body'].tolist() 
Y = Y['subreddit'].tolist()
Y_ = np.array(Y)
X_ = np.array(X)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
vectorizerfeature = CountVectorizer(min_df =1,binary=True,stop_words = my_stop_words, tokenizer=StemTokenizer())
vectors_train = vectorizerfeature.fit_transform(X_)
label = Y_

import math
Y_original = []
topics =['rpg', 'anime', 'datascience', 'hardware', 'cars', 'gamernews', 'gamedev', 'computers']
for i in range(vectors_train.todense().shape[0]):
  Y_original.append(topics.index(label[i]))
ll=np.zeros((vectors_train.todense().shape[1], len(topics)))
for i in range(len(topics)):
  for j in range(vectors_train.todense().shape[1]): #每个词
    x=vectors_train.todense()
    x0 = x[:, j]
    A=0
    B=0
    C=0
    D=0
    for jj in range(vectors_train.todense().shape[0]):
      if x0[jj]!=0:
        if Y_original[jj]==i:
          A=A+1
        else:
          B=B+1
      else:
        if Y_original[jj]==i:
          C=C+1
        else:
          D=D+1
    N=vectors_train.todense().shape[0]
    if A ==0 :
      ll[j][i]=0
    else:
      if B==0:
        ll[j][i]=0
      else:
        ll[j][i]=math.log(A/(B))
#print(ll)
llr=[]
for i in range((vectors_train.todense().shape[1])):
  max=np.max(ll[i, :])
  llr.append(max)
#print(llr)
precis = []
f1 = []
reca= []
for feanum in range(5):

  import heapq
  lis= llr.copy()
  n=(feanum+1)*1000
  max_number = heapq.nlargest(n, lis) 
  max_index = []
  for t in max_number:
      index = lis.index(t)
      max_index.append(index)
      lis[index] = 0
      
  #print(max_number)
  #print(max_index)

  voca=[]
  for i in max_index:
    voca.append(vectorizerfeature.get_feature_names()[i])
  #print(voca)
  voca2 = sorted(set(voca),key=voca.index)

  kf = KFold(n_splits=6,shuffle=True)
  kf.get_n_splits(X_)
  print(kf.split(X_))
  accuracy = []
  f1_sc = []
  recall_sc= []
  X_train_ = []
  y_train_ = []
  X_test_ = []
  y_test_ = []
  for train_index, test_index in kf.split(X_):

    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = Y_[train_index].tolist(), Y_[test_index].tolist()
    X_train_ = []
    X_test_ = []

    print(type(X_train))
    #print(y_train)
    X_train = X_train.tolist()
    X_test = X_test.tolist()
    vectorizer = CountVectorizer(vocabulary=voca2,binary=True,stop_words = my_stop_words,  tokenizer=StemTokenizer())
    vectors_train = vectorizer.fit_transform(X_train)
    vectors_test = vectorizer.transform(X_test)
    print(vectors_train.shape)
    trains = vectors_train.todense()
    tests = vectors_test.todense()
    print(len(tests))
    print(len(y_test))

    NBN1 = NBN(trains, y_train)
    topic_portion,freq_feature,topics = NBN1.build_NBN()
    #print(topics.tolist())
    #print(y_test)
    accu, result = Accu_eval(tests,y_test,topics)
    n=len(y_test)
    y_test0=[]
    for i in range(n):
      y_test0.append(topics.tolist().index([y_test[i]]))

    accuracy.append(metrics.precision_score(y_test0, result[0], average='weighted'))
    f1_sc.append(metrics.f1_score(y_test0, result[0], average='weighted'))
    recall_sc.append(metrics.recall_score(y_test0, result[0], average='weighted'))
    print(metrics.classification_report(y_test0, result[0]))

  print(np.mean(accuracy))
  print(np.mean(f1_sc))
  print(np.mean(recall_sc))
  precis.append(np.mean(accuracy))
  f1.append(np.mean(f1_sc))
  reca.append(np.mean(recall_sc))
print('precision_score',precis)
print('f1_score',f1)
print('recall_score',reca)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

<generator object _BaseKFold.split at 0x7fa408cd9200>
<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

(83, 10)
17
17
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.60      1.00      0.75         3
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         1
           4       0.30      1.00      0.46         3
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00         2

    accuracy                           0.47        17
   macro avg       0.41      0.50      0.41        17
weighted avg       0.34      0.47      0.35        17

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 

(84, 50)
16
16
              precision    recall  f1-score   support

           0       0.33      0.50      0.40         2
           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.36      1.00      0.53         4
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         4

    accuracy                           0.31        16
   macro avg       0.12      0.25      0.16        16
weighted avg       0.13      0.31      0.18        16

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 

(84, 50)
16
16
              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       1.00      0.67      0.80         3
           2       0.00      0.00      0.00         1
           4       0.50      1.00      0.67         5
           6       1.00      0.50      0.67         2

    accuracy                           0.69        16
   macro avg       0.70      0.55      0.58        16
weighted avg       0.78      0.69      0.68        16

0.5281098599113304
0.45037065071126486
0.5
[0.47140339576001344, 0.5830809033014915, 0.5903350734233087, 0.5142258986928104, 0.5281098599113304]
[0.44019980564098216, 0.4711370124973066, 0.4773919710428031, 0.43706680310270274, 0.45037065071126486]
[0.5189950980392156, 0.5189950980392156, 0.5202205882352942, 0.5006127450980392, 0.5]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


chi-squared

In [6]:
my_stop_words = text.ENGLISH_STOP_WORDS
#import datasets
#train = pd.read_csv('train_no_gamenews_computers.csv')
train = pd.read_csv('train.csv')
n=train.shape[0]
#print(n)
s=10000
skip = sorted(random.sample(range(1,n+1),n-s))
#print(len(skip))
train = pd.read_csv('train.csv',skiprows=skip)
test = pd.read_csv('test.csv')
#print(train.shape[0])
precis = []
f1 = []
reca= []
for feanum in range(5):
  X = pd.DataFrame(train['body'])
  Y = pd.DataFrame(train['subreddit'])
  X = X['body'].tolist() 
  Y = Y['subreddit'].tolist()
  Y_ = np.array(Y)
  X_ = np.array(X)
  from sklearn.feature_extraction.text import CountVectorizer
  from sklearn import metrics
  vectorizerfeature = CountVectorizer(min_df =1,binary=True,ngram_range=(1, 1),stop_words = my_stop_words, tokenizer=StemTokenizer())
  vectors_train = vectorizerfeature.fit_transform(X_)
  label = Y_.copy()

  from sklearn.feature_selection import SelectKBest
  from sklearn.feature_selection import chi2

  model1 = SelectKBest(chi2, k=(feanum+1)*1000)#选择k个最佳特征
  model1.fit_transform(vectors_train.todense(), label)
  x=model1.get_support(indices=True)
  x=x.tolist()
  voca=[]
  for i in x:
    voca.append(vectorizerfeature.get_feature_names()[i])

  voca2 = sorted(set(voca),key=voca.index)
  print(len(voca2))

  kf = KFold(n_splits=6,shuffle=True)
  kf.get_n_splits(X_)
  print(kf.split(X_))
  accuracy = []
  f1_sc = []
  recall_sc= []
  X_train_ = []
  y_train_ = []
  X_test_ = []
  y_test_ = []
  for train_index, test_index in kf.split(X_):

    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = Y_[train_index].tolist(), Y_[test_index].tolist()
    X_train_ = []
    X_test_ = []

    print(type(X_train))
    X_train = X_train.tolist()
    X_test = X_test.tolist()
    vectorizer = CountVectorizer(vocabulary=voca2,binary=True,ngram_range=(1, 2),stop_words = my_stop_words,  tokenizer=StemTokenizer())
    vectors_train = vectorizer.fit_transform(X_train)
    vectors_test = vectorizer.transform(X_test)
    print(vectors_train.shape)
    trains = vectors_train.todense()
    tests = vectors_test.todense()
    print(len(tests))
    print(len(y_test))

    NBN1 = NBN(trains, y_train)
    topic_portion,freq_feature,topics = NBN1.build_NBN()
    print(topics.tolist())
    accu, result = Accu_eval(tests,y_test,topics)
    n=len(y_test)
    y_test0=[]
    for i in range(n):
      y_test0.append(topics.tolist().index([y_test[i]]))
    accuracy.append(metrics.precision_score(y_test0, result[0], average='weighted'))
    f1_sc.append(metrics.f1_score(y_test0, result[0], average='weighted'))
    recall_sc.append(metrics.recall_score(y_test0, result[0], average='weighted'))
    print(metrics.classification_report(y_test0, result[0]))

  print(np.mean(accuracy))
  print(np.mean(f1_sc))
  print(np.mean(recall_sc))
  precis.append(np.mean(accuracy))
  f1.append(np.mean(f1_sc))
  reca.append(np.mean(recall_sc))
print('precision_score',precis)
print('f1_score',f1)
print('recall_score',reca)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

10
<generator object _BaseKFold.split at 0x7fa40a1689e8>
<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

(83, 10)
17
17
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           2       0.06      1.00      0.11         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         4

    accuracy                           0.06        17
   macro avg       0.01      0.17      0.02        17
weighted avg       0.00      0.06      0.01        17

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 

(84, 50)
16
16
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

           0       0.10      1.00      0.18         1
           1       0.00      0.00      0.00         1
           2       1.00      0.40      0.57         5
           3       0.00      0.00      0.00         2
           4       1.00      0.67      0.80         3
           5       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         1

    accuracy                           0.31        16
   macro avg       0.30      0.30      0.22        16
weighted avg       0.51      0.31      0.34        16

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 

(84, 50)
16
16
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

           0       0.43      0.75      0.55         4
           2       0.50      1.00      0.67         3
           3       0.00      0.00      0.00         3
           4       1.00      0.67      0.80         3
           6       0.00      0.00      0.00         2
           7       1.00      1.00      1.00         1

    accuracy                           0.56        16
   macro avg       0.49      0.57      0.50        16
weighted avg       0.45      0.56      0.47        16

0.34633307868601987
0.30913412567824333
0.36151960784313725
[0.02567492070357555, 0.032618313869665516, 0.2474470406563832, 0.32913978722802256, 0.34633307868601987]
[0.04358158468065588, 0.04872219355578489, 0.22995754717435654, 0.3077462366065307, 0.30913412567824333]
[0.15073529411764705, 0.15073529411764705, 0.32107843137254904, 0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


tfidf

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
my_stop_words = text.ENGLISH_STOP_WORDS
#import datasets
#train = pd.read_csv('train_no_gamenews_computers.csv')


precis = []
f1 = []
reca= []
for feanum in range(5):
  train = pd.read_csv('train.csv')
  n=train.shape[0]
  #print(n)
  s=10000
  maxFeatures=((feanum+1)*1000)
  skip = sorted(random.sample(range(1,n+1),n-s))
  #print(len(skip))
  train = pd.read_csv('train.csv',skiprows=skip)
  test = pd.read_csv('test.csv')
  #print(train.shape[0])

  X = pd.DataFrame(train['body'])
  Y = pd.DataFrame(train['subreddit'])
  X = X['body'].tolist() 
  Y = Y['subreddit'].tolist()
  Y_ = np.array(Y)
  X_ = np.array(X)

  # vectorizer = TfidfVectorizer(max_features=maxFeatures,stop_words = my_stop_words, tokenizer=StemTokenizer())
  # vectors_train = vectorizer.fit_transform(X_)
  # print(vectors_train.todense())
  # print(vectors_train.todense().shape)
  kf = KFold(n_splits=6,shuffle=True)
  kf.get_n_splits(X_)
  #print(kf.split(X_))
  accuracy = []
  f1_sc = []
  recall_sc= []
  X_train_ = []
  y_train_ = []
  X_test_ = []
  y_test_ = []
  for train_index, test_index in kf.split(X_):

    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = Y_[train_index].tolist(), Y_[test_index].tolist()
    X_train_ = []
    X_test_ = []

    print(type(X_train))
    X_train = X_train.tolist()
    X_test = X_test.tolist()
    vectorizer = TfidfVectorizer(max_features=maxFeatures,stop_words = my_stop_words, tokenizer=StemTokenizer())
    vectors_train = vectorizer.fit_transform(X_train)
    vectors_test = vectorizer.transform(X_test)
    print(vectors_train.shape)
    trains = vectors_train.todense()
    tests = vectors_test.todense()
    trains = trans_train(trains)
    tests = trans_train(tests)
    print(len(tests))
    print(len(y_test))

    NBN1 = NBN(trains, y_train)
    topic_portion,freq_feature,topics = NBN1.build_NBN()
    print(topics.tolist())
    accu, result = Accu_eval(tests,y_test,topics)
    n=len(y_test)
    y_test0=[]
    for i in range(n):
      y_test0.append(topics.tolist().index([y_test[i]]))
    accuracy.append(metrics.precision_score(y_test0, result[0], average='weighted'))
    f1_sc.append(metrics.f1_score(y_test0, result[0], average='weighted'))
    recall_sc.append(metrics.recall_score(y_test0, result[0], average='weighted'))
    print(metrics.classification_report(y_test0, result[0]))
  print(np.mean(accuracy))
  print(np.mean(f1_sc))
  print(np.mean(recall_sc))
  precis.append(np.mean(accuracy))
  f1.append(np.mean(f1_sc))
  reca.append(np.mean(recall_sc))
print('precision_score',precis)
print('f1_score',f1)
print('recall_score',reca)

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

(83, 10)
17
17
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         4
         1.0       0.50      0.17      0.25         6
         2.0       0.50      0.40      0.44         5
         3.0       0.00      0.00      0.00         0
         4.0       0.20      0.50      0.29         2
         5.0       0.00      0.00      0.00         0

    accuracy                           0.24        17
   macro avg       0.20      0.18      0.16        17
weighted avg       0.35      0.24      0.25        17

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyo

(83, 10)
17
17
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

         0.0       0.40      0.40      0.40         5
         1.0       0.00      0.00      0.00         0
         2.0       0.00      0.00      0.00         2
         3.0       0.00      0.00      0.00         2
         4.0       0.00      0.00      0.00         2
         5.0       0.00      0.00      0.00         3
         6.0       0.00      0.00      0.00         2
         7.0       0.00      0.00      0.00         1

    accuracy                           0.12        17
   macro avg       0.05      0.05      0.05        17
weighted avg       0.12      0.12      0.12        17

<class 'numpy.ndarray'>


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 

(84, 30)
16
16
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

         0.0       0.29      0.40      0.33         5
         1.0       0.00      0.00      0.00         2
         2.0       0.00      0.00      0.00         1
         3.0       0.00      0.00      0.00         2
         4.0       0.25      0.25      0.25         4
         5.0       0.00      0.00      0.00         0
         6.0       0.00      0.00      0.00         1
         7.0       0.00      0.00      0.00         1

    accuracy                           0.19        16
   macro avg       0.07      0.08      0.07        16
weighted avg       0.15      0.19      0.17        16

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarnin

(84, 30)
16
16
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

           0       0.33      0.20      0.25         5
           1       0.00      0.00      0.00         3
           2       0.25      0.50      0.33         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1

    accuracy                           0.12        16
   macro avg       0.08      0.10      0.08        16
weighted avg       0.14      0.12      0.12        16

0.25304913632119513
0.20374990273887336
0.21875
[0.29344362745098035, 0.40759220354808584, 0.25304913632119513]
[0.2801684562713974, 0.3465482026143791, 0.20374990273887336]
[0.3020833333333333, 0.3498774509803922, 0.21875]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


mutual information

In [15]:
my_stop_words = text.ENGLISH_STOP_WORDS
#import datasets
#train = pd.read_csv('train_no_gamenews_computers.csv')
train = pd.read_csv('train.csv')
n=train.shape[0]
#print(n)
s=10000
skip = sorted(random.sample(range(1,n+1),n-s))
#print(len(skip))
train = pd.read_csv('train.csv',skiprows=skip)
test = pd.read_csv('test.csv')
#print(train.shape[0])

X = pd.DataFrame(train['body'])
Y = pd.DataFrame(train['subreddit'])
X = X['body'].tolist() 
Y = Y['subreddit'].tolist()
Y_ = np.array(Y)
X_ = np.array(X)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
vectorizerfeature = CountVectorizer(min_df =1,binary=True,stop_words = my_stop_words, tokenizer=StemTokenizer())
vectors_train = vectorizerfeature.fit_transform(X_)
label = Y_
mutual_info=[]
for i in range(vectors_train.todense().shape[1]):
  x=vectors_train.todense()
  x0 = x[:, i]
  x0=np.array(x0).flatten()
  # 计算各特征与label的互信息
  mutual_info.append(metrics.mutual_info_score(x0, label))
#print(mutual_info)
precis = []
f1 = []
reca= []
for feanum in range(5):
  import heapq
  lis= mutual_info.copy()
  n=(feanum+1)*1000
  max_number = heapq.nlargest(n, lis) 
  max_index = []
  for t in max_number:
      index = lis.index(t)
      max_index.append(index)
      lis[index] = 0
      
  #print(max_number)
  #print(max_index)

  voca=[]
  for i in max_index:
    voca.append(vectorizerfeature.get_feature_names()[i])
  #print(voca)
  voca2 = sorted(set(voca),key=voca.index)

  vectorizer = CountVectorizer(vocabulary=voca2,binary=True,stop_words = my_stop_words, tokenizer=StemTokenizer())
  vectors_train = vectorizer.fit_transform(X_)
  vectors_train.todense()

  kf = KFold(n_splits=6,shuffle=True)
  kf.get_n_splits(X_)
  #print(kf.split(X_))
  accuracy = []
  f1_sc = []
  recall_sc= []
  X_train_ = []
  y_train_ = []
  X_test_ = []
  y_test_ = []
  for train_index, test_index in kf.split(X_):

    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = Y_[train_index].tolist(), Y_[test_index].tolist()
    X_train_ = []
    X_test_ = []

    print(type(X_train))
    X_train = X_train.tolist()
    X_test = X_test.tolist()
    vectorizer = CountVectorizer(vocabulary=voca2,binary=True, tokenizer=StemTokenizer())
    vectors_train = vectorizer.fit_transform(X_train)
    vectors_test = vectorizer.transform(X_test)
    print(vectors_train.shape)
    trains = vectors_train.todense()
    tests = vectors_test.todense()
    print(len(tests))
    print(len(y_test))

    NBN1 = NBN(trains, y_train)
    topic_portion,freq_feature,topics = NBN1.build_NBN()
    print(topics.tolist())
    accu, result = Accu_eval(tests,y_test,topics)
    n=len(y_test)
    y_test0=[]
    for i in range(n):
      y_test0.append(topics.tolist().index([y_test[i]]))
    accuracy.append(metrics.precision_score(y_test0, result[0], average='weighted'))
    f1_sc.append(metrics.f1_score(y_test0, result[0], average='weighted'))
    recall_sc.append(metrics.recall_score(y_test0, result[0], average='weighted'))
    print(metrics.classification_report(y_test0, result[0]))
  print(np.mean(accuracy))
  print(np.mean(f1_sc))
  print(np.mean(recall_sc))
  precis.append(np.mean(accuracy))
  f1.append(np.mean(f1_sc))
  reca.append(np.mean(recall_sc))
print('precision_score',precis)
print('f1_score',f1)
print('recall_score',reca)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

<class 'numpy.ndarray'>
(83, 10)
17
17
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

         0.0       0.40      0.40      0.40         5
         1.0       1.00      1.00      1.00         3
         2.0       0.00      0.00      0.00         2
         3.0       0.00      0.00      0.00         0
         4.0       1.00      0.67      0.80         3
         5.0       0.00      0.00      0.00         2
         6.0       0.00      0.00      0.00         2

    accuracy                           0.41        17
   macro avg       0.34      0.30      0.31        17
weighted avg       0.47      0.41      0.44        17

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarnin

(83, 10)
17
17
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

           0       0.33      0.50      0.40         2
           1       1.00      1.00      1.00         2
           2       0.83      1.00      0.91         5
           3       0.40      1.00      0.57         2
           4       1.00      1.00      1.00         1
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         1

    accuracy                           0.65        17
   macro avg       0.45      0.56      0.49        17
weighted avg       0.51      0.65      0.56        17

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(84, 50)
16
16
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

           0       0.40      1.00      0.57         2
           1       0.75      1.00      0.86         3
           2       1.00      1.00      1.00         3
           3       1.00      0.75      0.86         4
           4       0.00      0.00      0.00         2
           6       1.00      0.50      0.67         2

    accuracy                           0.75        16
   macro avg       0.69      0.71      0.66        16
weighted avg       0.75      0.75      0.72        16

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(84, 50)
16
16
[['rpg'], ['anime'], ['datascience'], ['hardware'], ['cars'], ['gamernews'], ['gamedev'], ['computers']]
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         3
           1       1.00      0.50      0.67         2
           2       1.00      0.75      0.86         4
           3       1.00      1.00      1.00         2
           4       0.50      1.00      0.67         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1

    accuracy                           0.69        16
   macro avg       0.50      0.53      0.48        16
weighted avg       0.66      0.69      0.63        16

0.705484068627451
0.655199933508757
0.6709558823529411
[0.5891499023851964, 0.6683239962651727, 0.6583566760037348, 0.6801937441643324, 0.705484068627451]
[0.54670417409388, 0.5904890996802762, 0.6247545482839602, 0.6354685143288

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
